In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
#匯入user.movie互動資料
rating = pd.read_csv("Yelp\\user_business.dat",sep = "\t",names=["userId","businessId","rating"])
print("互動資料:")
rating

互動資料:


,userId,businessId,rating
0,1,8391,5
1,1,8971,5
2,2,186,5
3,2,205,5
4,2,209,4
...,...,...,...
198392,16238,13256,5
198393,16239,2580,4
198394,16239,10525,1
198395,16239,12714,4


In [3]:
#data filtering (刪除互動<3的資料)
for number in range(1,16240):
    filt = (rating['userId'] == number)
    if len(rating.loc[filt])<3:
#         print(number,"/",len(rating.loc[filt]))
        rating =  rating.drop(index = rating[(rating.userId == number)].index.tolist())
rating

,userId,businessId,rating
2,2,186,5
3,2,205,5
4,2,209,4
5,2,461,3
6,2,499,2
...,...,...,...
198392,16238,13256,5
198393,16239,2580,4
198394,16239,10525,1
198395,16239,12714,4


In [5]:
#建立user-item矩陣
user_item = pd.pivot_table(rating,values='rating',columns='businessId',index='userId')
user_item

businessId,1,2,3,4,5,6,7,8,9,10,...,14275,14276,14277,14278,14279,14280,14281,14282,14283,14284
userId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
user_item=user_item.values
user_item = np.where(np.isnan(user_item), 0, user_item)
user_item

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
#分割資料集
train_data, test_data = train_test_split(user_item, random_state=None, train_size=0.8)
print("Train data:")
train_data

Train data:


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
print("Test data:")
test_data

Test data:


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
# Define the number of folds (K) for cross-validation
k = 5

# Define lists to store the evaluation metrics for each fold
rmse_scores = []
mae_scores = []

# Perform K-fold Cross-Validation
kf = KFold(n_splits=k)

In [17]:
for train_index, test_index in kf.split(train_data):
    # Split the data into train and test sets for the current fold
    train_ratings, test_ratings = user_item[train_index], user_item[test_index]

    # Data preprocessing (do normalized)
    normalized_Train_data = train_ratings / np.max(train_ratings)
    normalized_Test_data = test_ratings / np.max(test_ratings)
    
    # Define and compile your model
    model = Sequential()
    model.add(Dense(400, activation='tanh', input_dim=user_item.shape[1]))
    model.add(Dense(200, activation='tanh'))
    model.add(Dense(user_item.shape[1], activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam')


    # Train the model
    model.fit(normalized_Train_data, normalized_Train_data, epochs=3, batch_size=128, verbose=1)

    # Make predictions on the test set
    predicted_ratings = model.predict(normalized_Test_data)
    print(predicted_ratings)

    # Calculate evaluation metrics (RMSE and MAE)
    true_ratings = test_ratings.flatten()
    predicted_ratings = predicted_ratings.flatten()

    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    mae = np.mean(np.abs(true_ratings - predicted_ratings))

    # Append the evaluation scores to the lists
    rmse_scores.append(rmse)
    mae_scores.append(mae)

Epoch 1/3
43/43 [==============================] - 8s 74ms/step - loss: 0.1225
Epoch 2/3
43/43 [==============================] - 3s 73ms/step - loss: 0.0011
Epoch 3/3
43/43 [==============================] - 4s 7ms/step
[[0.00581336 0.00592914 0.00608365 ... 0.00623727 0.00524066 0.00654231]
 [0.00584916 0.00596757 0.00612042 ... 0.00627561 0.00527519 0.00658114]
 [0.00766552 0.00786174 0.00799464 ... 0.00820495 0.00698309 0.00856824]
 ...
 [0.00872475 0.00895494 0.00908535 ... 0.009321   0.00798796 0.00972155]
 [0.01192224 0.01226161 0.0123659  ... 0.01271041 0.01101046 0.01318282]
 [0.00797633 0.00817867 0.0083154  ... 0.0085348  0.0072811  0.00891724]]
Epoch 1/3
43/43 [==============================] - 7s 73ms/step - loss: 0.1226
Epoch 2/3
43/43 [==============================] - 3s 72ms/step - loss: 0.0011
Epoch 3/3
43/43 [==============================] - 3s 7ms/step
[[0.01054702 0.00957954 0.01064952 ... 0.00997956 0.01026735 0.00992817]
 [0.0062549  0.0056261  0.00634952 ... 0.

In [18]:
# Calculate and print the average evaluation scores
avg_rmse = np.mean(rmse_scores)
# avg_mae = np.mean(mae_scores)
print("Average RMSE:", avg_rmse)
# print("Average MAE:", avg_mae)

Average RMSE: 0.1546459734325435


In [19]:
predicted_ratings = model.predict(test_data)
predicted_ratings

54/54 [==============================] - 3s 7ms/step


array([[0.00643021, 0.00630514, 0.00641136, ..., 0.00567661, 0.00628649,
        0.00611864],
       [0.00748884, 0.00739092, 0.0074682 , ..., 0.00663595, 0.00734   ,
        0.00714515],
       [0.01306084, 0.0130303 , 0.0129845 , ..., 0.01178524, 0.01291778,
        0.01259764],
       ...,
       [0.00642018, 0.0062942 , 0.00640114, ..., 0.00566738, 0.00627594,
        0.00610876],
       [0.00642024, 0.00629425, 0.00640119, ..., 0.00566742, 0.006276  ,
        0.0061088 ],
       [0.0065893 , 0.00646897, 0.00656689, ..., 0.00581979, 0.00644674,
        0.0062718 ]], dtype=float32)

In [20]:
print(len(predicted_ratings[0]))
print( len(test_data[0]))

14174
14174


In [21]:
# A=[]
# A=predicted_ratings[1]
print(type(predicted_ratings))
predicted_ratings

<class 'numpy.ndarray'>


array([[0.00643021, 0.00630514, 0.00641136, ..., 0.00567661, 0.00628649,
        0.00611864],
       [0.00748884, 0.00739092, 0.0074682 , ..., 0.00663595, 0.00734   ,
        0.00714515],
       [0.01306084, 0.0130303 , 0.0129845 , ..., 0.01178524, 0.01291778,
        0.01259764],
       ...,
       [0.00642018, 0.0062942 , 0.00640114, ..., 0.00566738, 0.00627594,
        0.00610876],
       [0.00642024, 0.00629425, 0.00640119, ..., 0.00566742, 0.006276  ,
        0.0061088 ],
       [0.0065893 , 0.00646897, 0.00656689, ..., 0.00581979, 0.00644674,
        0.0062718 ]], dtype=float32)

In [22]:
from sklearn.metrics import ndcg_score
k = 10  # Top k 推薦項目數量

ndcg_scores = []

for i in range(len(test_data)):
    true_labels = test_data[i]
    predicted_labels = predicted_ratings[i]

    # 按照預測評分降續排序，提取前k個項目的真實評分
    top_k_indices = np.argsort(predicted_labels)[::-1][:k]
    true_ratings_k = true_labels[top_k_indices]

    # 計算累積增益
    cumulative_gain = (2 ** true_ratings_k - 1) / np.log2(np.arange(2, k + 2))

    # 對真實評分降續排序，提取前k個項目的真實評分
    ideal_true_ratings_k = np.sort(true_labels)[::-1][:k]

    # 計算理想累積增益
    ideal_cumulative_gain = (2 ** ideal_true_ratings_k - 1) / np.log2(np.arange(2, k + 2))

    # 計算NDCG@k
    ndcg_at_k = np.sum(cumulative_gain) / np.sum(ideal_cumulative_gain)

    ndcg_scores.append(ndcg_at_k)

# 計算平均NDCG@k
average_ndcg_at_k = np.mean(ndcg_scores)

print("Average NDCG@{}: {:.4f}".format(k, average_ndcg_at_k))

Average NDCG@10: 0.0463


In [23]:
min_rating = 0
max_rating = 5

In [24]:
k = 10  # Top k 推薦項目數量
threshold = 0.5  

recall_scores = []

for i in range(len(test_data)):
    true_ratings = test_data[i]
    predicted_values = predicted_ratings[i] * (max_rating - min_rating) + min_rating

    # 預測預測評分中前k個最高評分的項目索引
    top_k_indices = predicted_values.argsort()[-k:][::-1]
#     print("預測的movies:")
#     print(top_k_indices)

    # 計算真實評分中大於等於threshold的項目數量
    relevant_items = np.sum(true_ratings >= threshold)

    # 計算推薦項目中真實評分大於等於threshold的項目數量
    recommended_relevant_items = np.sum(true_ratings[top_k_indices] >= threshold)

    # 計算 Recall@k
    recall_at_k = recommended_relevant_items / relevant_items

    recall_scores.append(recall_at_k)

# 計算平均 Recall@k
average_recall_at_k = np.mean(recall_scores)

print("Average Recall@{}: {:.4f}".format(k, average_recall_at_k))


Average Recall@10: 0.0369
